# Claim to support / refute:

#### - “The   financial   markets   do   not   punish   security   breaches.”

## Organizing & Cleaning the Data

In [1]:
# Imports

import pandas as pd
import xlrd as xl

In [2]:
# Load data into Excel file
data = pd.ExcelFile("InfoisBeautiful-DataBreaches.xlsx")

In [3]:
# See sheet names in excel file
data.sheet_names

['2017 update', 'Jan 2015 update', 'July 2013 update (old)']

In [4]:
# Load first sheet into dataframe

DF = data.parse('2017 update')

In [5]:
# Preview data columns
DF.columns

Index(['Entity', 'alternative name', 'story', 'YEAR', 'records lost',
       'ORGANISATION', 'METHOD OF LEAK', 'interesting story',
       'NO OF RECORDS STOLEN', 'DATA SENSITIVITY', 'UNUSED', 'UNUSED.1',
       'Exclude', 'Unnamed: 13', '1st source link', '2nd source link',
       '3rd source', 'source name'],
      dtype='object')

In [6]:
# Remove unnecessary columns

StripCols = DF[["Entity", "YEAR", "records lost", "ORGANISATION", "METHOD OF LEAK", "DATA SENSITIVITY"]]
StripCols.head()

,Entity,YEAR,records lost,ORGANISATION,METHOD OF LEAK,DATA SENSITIVITY
0,NaN,"years are encoded (0=2004, 8 = 2012, 9 = 2013,...","(use 3m, 4m, 5m or 10m to approximate unknown ...",NaN,NaN,1. Just email address/Online information 20 SS...
1,AOL,0,92000000,web,inside job,1
2,Automatic Data Processing,1,125000,financial,poor security,20
3,Ameritrade Inc.,1,200000,financial,lost / stolen device,20
4,Citigroup,1,3900000,financial,lost / stolen device,300


In [7]:
# remove first row with NaN values

cleaned = StripCols.dropna()
cleaned.head()

,Entity,YEAR,records lost,ORGANISATION,METHOD OF LEAK,DATA SENSITIVITY
1,AOL,0,92000000,web,inside job,1
2,Automatic Data Processing,1,125000,financial,poor security,20
3,Ameritrade Inc.,1,200000,financial,lost / stolen device,20
4,Citigroup,1,3900000,financial,lost / stolen device,300
5,Cardsystems Solutions Inc.,1,40000000,financial,hacked,300


Convert year values to real years

In [8]:
cleaned['YEAR'].astype(int)   # Convert data type
cleaned2 = cleaned.copy()
cleaned2['Real Year'] = cleaned['YEAR'] + 2004

cleaned3 = cleaned2[["Entity", "Real Year", "records lost", "ORGANISATION", "METHOD OF LEAK", "DATA SENSITIVITY"]]
cleaned3.tail()

,Entity,Real Year,records lost,ORGANISATION,METHOD OF LEAK,DATA SENSITIVITY
249,CEX,2018,2000000,retail,accidentally published,300
250,Swedish Transport Agency,2018,3000000,government,poor security,50000
251,Instagram,2018,6000000,web,hacked,1
252,Equifax,2018,143000000,financial,hacked,50000
253,Spambot,2018,711000000,web,poor security,4000


#### Some data in various rows had errors and need altering for consistency:

Most recent breaches appear as "2018". Change to "2017".

In [9]:
cleaned4 = cleaned3.copy()
cleaned4.loc[cleaned3['Real Year'] > 2017, 'Real Year'] = 2017
cleaned4.tail()

,Entity,Real Year,records lost,ORGANISATION,METHOD OF LEAK,DATA SENSITIVITY
249,CEX,2017,2000000,retail,accidentally published,300
250,Swedish Transport Agency,2017,3000000,government,poor security,50000
251,Instagram,2017,6000000,web,hacked,1
252,Equifax,2017,143000000,financial,hacked,50000
253,Spambot,2017,711000000,web,poor security,4000


One occurence of "web, tech" should change to "tech, web" for consistency with other "tech, web" rows

In [10]:
cleaned5 = cleaned4.copy()
cleaned5.loc[cleaned4['ORGANISATION'] == 'web, tech', 'ORGANISATION'] = 'tech, web'

Change one occurence of data sensitivity "3" to "300"

In [11]:
cleaned6 = cleaned5.copy()
cleaned6.loc[cleaned5['DATA SENSITIVITY'] == 3, 'DATA SENSITIVITY'] = 300

"twitch.tv" organization is listed as "healthcare". Change to "web, gaming"

In [12]:
cleaned7 = cleaned6.copy()
cleaned7.loc[cleaned6['Entity'] == 'Twitch.tv', 'ORGANISATION'] = 'gaming'

In [13]:
cleaned7.loc[cleaned7['Entity'] == 'Twitch.tv']

,Entity,Real Year,records lost,ORGANISATION,METHOD OF LEAK,DATA SENSITIVITY
180,Twitch.tv,2014,10000000,gaming,hacked,1


## Gathering stock market data for each entity

#### Gather list of company stock ticker symbols:

In [51]:
import re
from fuzzywuzzy import string_processing       # fuzzywuzzy package used to manage ascii / unicode errors
from fuzzywuzzy import utils

# New column for ticker names
cleaned7['Stock Ticker'] = 'N/A'

    # All Entity names to a list
names = cleaned7.Entity.unique().tolist()
    # Remove ASCII characters from names
names2 = []

    # cleaning names for higher chance of successful search query
for i in names:
 i = i.replace("\"", "")
 i = i.replace("\'", "")
 i = i.replace(",", " ")
 i = utils.full_process(i)
 names2.append(i)
                        # Dictionaries:
Original = {}       # Original names from raw data
RealNames = {}      # Names w/ special characters removed
WebNames = {}       # Names for entering into URL

     # non-breaking space for web search: '%20' or '&nbsp;'
for i in range(0, len(names)):
 Original[i] = names[i]              # Need this to match back to DF
 RealNames[i] = names2[i]
 WebNames[i] = names2[i].replace(' ', '&nbsp;')

Find ticker symbols of each company and put into new column

In [52]:
import urllib.request

for i in RealNames:
                            # Perform the search query
    webLink = 'http://d.yimg.com/autoc.finance.yahoo.com/autoc?query=' + WebNames[i] + '&region=1&lang=en'
    with urllib.request.urlopen(webLink) as response:
       html = response.read()
    
    Buis = str(html)        # Convert byte type to string
    
                            # Result if query returns nothing
    empty = len(RealNames[i]) + len('{"ResultSet":{"Query":"","Result":[]}}')

    Ticker = []             # List of tickers from search query

    if len(Buis) > empty:             # If search query finds any info,
        BuisInfo = Buis.split(',')    # Split string into array

        for entry in BuisInfo:
            if "symbol" in entry:     # Append ticker symbols to list
                symbol = entry.split("\"")     # extract ticker symbol
                Ticker.append(symbol[-2])
    if len(Ticker) > 0:
        result = min(Ticker, key=len)
        cleaned7['Stock Ticker'][cleaned7['Entity'] == Original[i]] = result

Number of rows that did not get a stock ticker:

In [53]:
cleaned7['Entity'][cleaned7['Stock Ticker'] == 'N/A'].count()

69

#### Gathering stock price data for the year each company had data breach:

In [59]:
from pandas_datareader import data, wb
import pandas_datareader.data as web
import datetime

In [68]:
start = datetime.datetime(2004, 1, 1)
end = datetime.datetime(2004,12,31)
f = web.DataReader("AAPL", "yahoo", start, end)
f

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2004-01-02,1.539286,1.553571,1.512857,1.520000,1.367477,36160600
2004-01-05,1.530000,1.599286,1.530000,1.583571,1.424670,98754600
2004-01-06,1.589286,1.601429,1.550714,1.577857,1.419529,127337000
2004-01-07,1.578571,1.630714,1.566429,1.613571,1.451659,146718600
2004-01-08,1.631429,1.695000,1.617857,1.668571,1.501141,115075800
2004-01-09,1.659286,1.723571,1.627857,1.642857,1.478006,106864800
2004-01-12,1.660714,1.714286,1.650000,1.695000,1.524917,121886800
2004-01-13,1.764286,1.774286,1.704286,1.722857,1.549979,169754200
2004-01-14,1.742857,1.752857,1.698571,1.728571,1.555120,155010800


In [67]:
cleaned7.loc[cleaned7['Entity'] == 'Apple']

,Entity,Real Year,records lost,ORGANISATION,METHOD OF LEAK,DATA SENSITIVITY,Stock Ticker
146,Apple,2013,275000,tech,hacked,1,AAPL


In [ ]:
# Export to CSV

cleaned7.to_csv("DataBreaches.csv", header=True, encoding='utf-8')